In [1]:
import notebook_importer

In [2]:
from pond.Tensor import PrivateTensor, NumpyTensor
from pond.Neural_Nets import Sequential, Dense, Sigmoid, Diff, Reveal #, Softmax, CrossEntropy

importing notebook from /Users/dahl/work/mortendahl/privateml/image-analysis/pond/Tensor.ipynb
importing notebook from /Users/dahl/work/mortendahl/privateml/image-analysis/pond/Neural Nets.ipynb


In [3]:
import numpy as np

# Xor

In [4]:
model = Sequential([
    Dense(4, 2),
    Sigmoid(),
    Dense(1, 4),
    Sigmoid()
])

In [5]:
x = NumpyTensor.load(np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
]))

y = NumpyTensor.load(np.array([[
    0,
    0,
    1,
    1
]]).T)

model.initialize()

model.fit(x, y, Diff(), epochs=1000)

model.predict(x)

NumpyTensor([[ 0.15162684]
 [ 0.11963637]
 [ 0.87227455]
 [ 0.84987141]])

In [6]:
x = PrivateTensor.load(np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
]))

y = PrivateTensor.load(np.array([[
    0,
    0,
    1,
    1
]]).T)

model.initialize()

model.fit(x, y, Diff(), epochs=1000)

model.predict(x)

PrivateTensor([[ 0.18483634]
 [ 0.14544616]
 [ 0.86051134]
 [ 0.82967177]])

# MNIST

In [7]:
import keras
from keras.utils import to_categorical

Using TensorFlow backend.


In [8]:
def preprocess_data(dataset):
    
    (x_train, y_train), (x_test, y_test) = dataset
    
    # NOTE: this is the shape used by Tensorflow; other backends may differ
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test  = x_test.reshape(x_test.shape[0], 28, 28, 1)
    
    x_train  = x_train.astype('float32')
    x_test   = x_test.astype('float32')
    x_train /= 255
    x_test  /= 255

    y_train = to_categorical(y_train, 5)
    y_test  = to_categorical(y_test, 5)
    
    return (x_train, y_train), (x_test, y_test)

In [9]:
def load_data():
    
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    x_train_public = x_train[y_train < 5]
    y_train_public = y_train[y_train < 5]
    x_test_public  = x_test[y_test < 5]
    y_test_public  = y_test[y_test < 5]
    public_dataset = (x_train_public, y_train_public), (x_test_public, y_test_public)

    x_train_private = x_train[y_train >= 5]
    y_train_private = y_train[y_train >= 5] - 5
    x_test_private  = x_test[y_test >= 5]
    y_test_private  = y_test[y_test >= 5] - 5
    private_dataset = (x_train_private, y_train_private), (x_test_private, y_test_private)
    
    return preprocess_data(public_dataset), preprocess_data(private_dataset)

### Pre-train on public data

In [10]:
public_dataset, private_dataset = load_data()

feature_layers = [
    keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(28, 28, 1)),
    keras.layers.Activation('sigmoid'),
    keras.layers.Conv2D(32, (3, 3), padding='same'),
    keras.layers.Activation('sigmoid'),
    keras.layers.AveragePooling2D(pool_size=(2,2)),
    keras.layers.Dropout(.25),
    keras.layers.Flatten()
]

classification_layers = [
    keras.layers.Dense(128),
    keras.layers.Activation('sigmoid'),
    keras.layers.Dropout(.50),
    keras.layers.Dense(5),
    keras.layers.Activation('softmax')
]

model = keras.models.Sequential(feature_layers + classification_layers)

# pre-train on public dataset

(x_train, y_train), (x_test, y_test) = public_dataset

# TODO don't crop
x_train = x_train[:10]
y_train = y_train[:10]
x_test = x_test[:10]
y_test = y_test[:10]

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

model.fit(
    x_train, y_train,
    epochs=1,
    batch_size=32,
    verbose=1,
    validation_data=(x_test, y_test))

Train on 10 samples, validate on 10 samples
Epoch 1/1
10/10 [==============================] - 0s - loss: 2.3186 - acc: 0.0000e+00 - val_loss: 1.9367 - val_acc: 0.3000


### Extract features from private data (unencrypted)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
__________

In [12]:
flatten_layer = model.get_layer(index=7)
assert flatten_layer.name.startswith('flatten_')

extractor = keras.models.Model(
    inputs=model.input, 
    outputs=flatten_layer.output
)

In [13]:
(x_train, y_train), (x_test, y_test) = private_dataset

# TODO don't crop
x_train = x_train[:10]
y_train = y_train[:10]

features = extractor.predict(x_train)

### Fine-tune on encrypted data

In [14]:
class Sigmoid:
    
    def __init__(self):
        self.cache = None
        self.max = 0
        self.min = 0
    
    def initialize(self):
        pass
    
    def forward(self, x):
        self.max = max(self.max, x.values.max())
        self.min = min(self.min, x.values.min())
        y = NumpyTensor.load(1. / (x.neg().exp() + 1).values)
        self.cache = y
        return y
    
    def backward(self, d_out):
        out = self.cache
        d_x = d_out * out * (out.neg() + 1)
        return d_x

In [15]:
class Softmax:
    
    def __init__(self):
        self.cache = None
    
    def initialize(self):
        pass
    
    def forward(self, scores):
        likelihoods = scores.exp()
        probs = likelihoods.div(likelihoods.sum1())
        self.cache = probs
        return probs
    
    def backward(self, d_probs):
        # TODO does the split between Softmax and CrossEntropy make sense?
        probs = self.cache
        batch_size = probs.shape[0]
        d_scores = probs - d_probs
        d_scores = d_scores.div(batch_size)
        return d_scores

In [16]:
class CrossEntropy:
    
    def evaluate(self, probs_pred, probs_correct):
        batch_size = probs_pred.shape[0]
        losses = (probs_pred * probs_correct).log().neg()
        loss = losses.sum0().div(batch_size)
        return loss
        
    def derive(self, y_pred, y_correct):
        return y_correct

In [17]:
classifier = Sequential([
    Dense(128, 6272),
    Sigmoid(),
    Dense(5, 128),
    Reveal(),
    Softmax()
])

In [18]:
classifier.initialize()
classifier.fit(
    NumpyTensor.load(features), 
    NumpyTensor.load(y_train), 
    loss=CrossEntropy(), 
    epochs=10
)

In [19]:
print(classifier.layers[1].min, classifier.layers[1].max)

-10.5452904156 10.7005468223


In [20]:
x = NumpyTensor.load(features)
print(x.values.min(), x.values.max())
x = classifier.layers[0].forward(x)
print(x.values.min(), x.values.max())
x = classifier.layers[1].forward(x)
print(x.values.min(), x.values.max())
x = classifier.layers[2].forward(x)
print(x.values.min(), x.values.max())
x = classifier.layers[3].forward(x)
print(x.values.min(), x.values.max())

0.282031 0.776631
-10.5451440463 10.7004484815
2.63202917464e-05 0.999977465679
-0.632343626571 0.277420840558
-0.632343626571 0.277420840558


In [21]:
classifier.initialize()
classifier.fit(
    PrivateTensor.load(features),
    NumpyTensor.load(y_train), 
    loss=CrossEntropy(), 
    epochs=10
)

AttributeError: 'PrivateTensor' object has no attribute 'values'